In [1]:
import pathlib
import cv2
import numpy as np
import pandas as pd
import ast

import matplotlib.pyplot as plt
from skimage import img_as_float

def get_index_of_file(file_path, index):
    list_of_files = sorted(pathlib.Path(file_path).iterdir())
    f = list_of_files[index]
    try:
        f = int(f.stem)
    except ValueError:
        for current_file in list_of_files:
            if current_file.name[0] != '.':
                f = current_file
                break
        f = int(f.stem)
    return f

def get_files_list(file_path, transition_frames):
    files_list = list()
    start_int = get_index_of_file(file_path, 0)
    for i in transition_frames:
        files_list.append(str(pathlib.Path(file_path)
                      .joinpath('{:03d}.jpg'.format(i+start_int))))
    return files_list

def make_summary(image_list):
    images = []
    scaling = 0.2
    for image_file in image_list:
        img = cv2.imread(image_file, flags=cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, None,
                         fx=scaling, fy=scaling, 
                         interpolation=cv2.INTER_AREA)
        images.append(img)
        
    summary_image = np.concatenate(np.array(images), axis=1)
    return summary_image

def process_date(folder_date, t_frames_list):
    files_list = get_files_list(folder_date, t_frames_list)
    summary_img = make_summary(files_list)
    image_name = '/Volumes/NO NAME/sleeper/summary_images/{}.jpg'.format(folder_date.stem)
    cv2.imwrite(image_name,
               summary_img)
    
def reduce_continued_movements(image_list):
    reduced_list = []
    for i, f in enumerate(image_list):
        if i == 0:
            continue
        if (image_list[i] - image_list[i-1] > 2):
            reduced_list.append(image_list[i-1])
        else:
            continue
    if image_list[-1] - image_list[-2] > 2:
        reduced_list.append(image_list[-1])
    return reduced_list

def process_all(record, frames_list_str='reduced_std_frames'):
    for index, row in record.iterrows():
        dirpath = '/Volumes/NO NAME/sleeper/photos_processed/'
        p = pathlib.Path(dirpath)
        folder = p.joinpath(row['folder_name'])
        t_frames_list = row[frames_list_str]
        process_date(folder, t_frames_list)

In [3]:
record = pd.read_csv('/Volumes/NO NAME/sleeper/record.csv', index_col=0)
record['t_frames_list'] = record['t_frames_list'].apply(lambda x: ast.literal_eval(x))
record['std_frames_list'] = record['std_frames_list'].apply(lambda x: ast.literal_eval(x))

record['reduced_t_frames'] = (record['t_frames_list']
                               .apply(lambda x: reduce_continued_movements(x)))
record['count_reduced_t_frames'] = (record['reduced_t_frames']
                               .apply(lambda x: len(x)))
record['reduced_std_frames'] = (record['std_frames_list']
                               .apply(lambda x: reduce_continued_movements(x)))
record['count_reduced_std_frames'] = (record['reduced_std_frames']
                               .apply(lambda x: len(x)))
record.to_csv('/Volumes/NO NAME/sleeper/record.csv')
record.tail(2)

,folder_name,duration,count_t_frames,t_frames_list,count_std_frames,std_frames_list,reduced_t_frames,count_reduced_t_frames,reduced_std_frames,count_reduced_std_frames
60,2019-05-07-20,274,19,"[53, 80, 95, 114, 127, 139, 208, 209, 210, 211...",8,"[208, 209, 250, 262, 263, 264, 266, 267]","[53, 80, 95, 114, 127, 139, 211, 232, 250, 267]",10,"[209, 250]",2
61,2019-05-08-21,257,22,"[0, 38, 39, 40, 42, 83, 119, 120, 126, 172, 17...",9,"[39, 40, 42, 119, 120, 173, 198, 210, 216]","[0, 42, 83, 120, 126, 173, 198, 210, 217, 234,...",11,"[42, 120, 173, 198, 210, 216]",6


In [192]:
process_all(record, 'reduced_std_frames')

In [197]:
record[record['folder_name']=="2019-04-07"]
record.loc[37, 'reduced_std_frames']

[24, 46, 54, 62, 71, 99, 121, 150, 161, 195, 217, 251, 261, 277, 300, 331, 335]